# Land cover classification at the Mississppi Delta

In this notebook, you will use a k-means **unsupervised** clustering
algorithm to group pixels by similar spectral signatures. **k-means** is
an **exploratory** method for finding patterns in data. Because it is
unsupervised, you don’t need any training data for the model. You also
can’t measure how well it “performs” because the clusters will not
correspond to any particular land cover class. However, we expect at
least some of the clusters to be identifiable as different types of land
cover.

You will use the [harmonized Sentinal/Landsat multispectral
dataset](https://lpdaac.usgs.gov/documents/1698/HLS_User_Guide_V2.pdf).
You can access the data with an [Earthdata
account](https://www.earthdata.nasa.gov/learn/get-started) and the
[`earthaccess` library from
NSIDC](https://github.com/nsidc/earthaccess):

## STEP 1: SET UP

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Import all libraries you will need for this analysis</li>
<li>Configure GDAL parameters to help avoid connection errors:
<code>python      os.environ["GDAL_HTTP_MAX_RETRY"] = "5"      os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"</code></li>
</ol></div></div>

In [2]:
#Set up analysis Part 1 of 3
# Import libraries / packages

# Basic Packages
import os # Reproducible file paths
from glob import glob # Find files by pattern
import pathlib # Find the home folder
import pickle # Serializing/unserializing objects
import re # Regular expressions
import warnings # Filter warning messages
import zipfile # Work with zip files

# Imports for spatial data
import cartopy.crs as ccrs # CRSs (Coordinate Reference Systems)
import earthaccess # Access NASA data from the cloud
import earthpy as et # Spatial data analysis
import geopandas as gpd # Work with vector data
import geoviews as gv # Holoviews extension for data visualization
import hvplot.pandas # Interactive tabular and vector data
import hvplot.xarray # Interactive raster
import numpy as np # Adjust images 
import pandas as pd # Group and aggregate
import rioxarray as rxr # Work with geospatial raster data
import rioxarray.merge as rxrmerge # Merge rasters
from tqdm.notebook import tqdm # Visualize progress of iterative operations
import xarray as xr # Adjust images
from shapely.geometry import Polygon # Keep track of polygons geometry
from sklearn.cluster import KMeans # kmeans clustering
from sklearn.decomposition import PCA # Principal component analysis
from sklearn.metrics import silhouette_score # Performance metrics
from sklearn.preprocessing import StandardScaler # Standardize features

# Import to be able to save plots
import holoviews as hv # be able to save hvplots

# Prevent GDAL from quitting due to momentary disruptions
os.environ["GDAL_HTTP_MAX_RETRY"] = "5"
os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"

# Suppress third party warnings - 'ignore'
warnings.simplefilter('ignore')

In [2]:
# Set Up Analysis Part 2 of 3

# Define and create the project data directory
landcover_cluster_data_dir = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'land_cover_clustering'
)
# Make the directory
os.makedirs(landcover_cluster_data_dir, 
            # If directory already exists no error is raised
            exist_ok=True)

# Call the data directory to check its location
landcover_cluster_data_dir

'/Users/briannagleason/earth-analytics/land_cover_clustering'

In [3]:
# Set Up Analysis Part 3 of 3

# Define and create nested path for watershed boundaries and HLS data
# Path for watershed boundaries
#watershed_bd_dir = os.path. join(
    #landcover_cluster_data_dir, 'watershed_boundary')
# Path for HLS data
#hls_dir = os.path.join(landcover_cluster_data_dir, 'hls')

# Create the directories for these paths 
#for a_dir in [watershed_bd_dir, hls_dir]:
    # Conditional statement if the path exists 
    #if not os.path.exists(a_dir):
        # Make a directory
        #os.makedirs(a_dir)


Below you can find code for a caching **decorator** which you can use in
your code. To use the decorator:

``` python
@cached(key, override)
def do_something(*args, **kwargs):
    ...
    return item_to_cache
```

This decorator will **pickle** the results of running the
`do_something()` function, and only run the code if the results don’t
already exist. To override the caching, for example temporarily after
making changes to your code, set `override=True`. Note that to use the
caching decorator, you must write your own function to perform each
task!

In [4]:
# Create a series of nested functions for caching a decorator

# Define function for the decorator
def cached(func_key, override=False):
    """
    A decorator to cache function results
    
    Parameters
    ==========
    key: str
      File basename used to save pickled results
    override: bool
      When True, re-compute even if the results are already stored
    """
    # Wrap caching function
    def compute_and_cache_decorator(compute_function):
        """
        Wrap the caching function
        
        Parameters
        ==========
        compute_function: function
          The function to run and cache results
        """
        # Computation and cache or load cached result
        def compute_and_cache(*args, **kwargs):
            """
            Perform a computation and cache, or load cached result.
            
            Parameters
            ==========
            args
              Positional arguments for the compute function
            kwargs
              Keyword arguments for the compute function
            """

            # Pickling and unpickling

            # Add an identifier from the particular function call
            # If 'cache_key' is found in kwargs
            if 'cache_key' in kwargs:
                # Create key by concatenating the 'func_key'
                key = '_'.join((func_key, kwargs['cache_key']))
            # If no'cache_key' is found in the kwargs    
            else:
                # Key will be set to the func_key
                key = func_key

            # Create file path for saving the cache
            path = os.path.join(
                # Specify root directory and location
                et.io.HOME, et.io.DATA_NAME, 'jars',
                # Pickle file named using 'key' 
                f'{key}.pickle')
            
            # Check if the cache exists already or override caching
            # If cache does not exist
            if not os.path.exists(path) or override:
                # Make jars directory if needed
                os.makedirs(os.path.dirname(path), 
                # If directory already exists no error is raised            
                exist_ok=True)
                # Run the compute function 
                result = compute_function(*args, **kwargs)
                # Pickle the object and save it in a file
                with open(path, 'wb') as file:
                    pickle.dump(result, file)

            # If cache already exists   
            else:
                # Unpickle the object
                with open(path, 'rb') as file:
                    # Load cached result
                    result = pickle.load(file)
            
            # Return innermost function for computation and cache    
            return result
        
        # Return wrap caching function
        return compute_and_cache
    
    # Return decorator to cache function results
    return compute_and_cache_decorator

## STEP 2: STUDY SITE

For this analysis, you will use a watershed from the [Water Boundary
Dataset](https://www.usgs.gov/national-hydrography/access-national-hydrography-products),
HU12 watersheds (WBDHU12.shp).

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Download the Water Boundary Dataset for region 8 (Mississippi)</li>
<li>Select watershed 080902030506</li>
<li>Generate a site map of the watershed</li>
</ol>
<p>Try to use the <strong>caching decorator</strong></p></div></div>

We chose this watershed because it covers parts of New Orleans an is
near the Mississippi Delta. Deltas are boundary areas between the land
and the ocean, and as a result tend to contain a rich variety of
different land cover and land use types.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-response"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div></div><div class="callout-body-container callout-body"><p>Write a 2-3 sentence <strong>site description</strong> (with
citations) of this area that helps to put your analysis in context.</p></div></div>

In [ ]:
# Pasted URL of watershead boundaries to refer back to
# https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/WBD/HU2/Shape/WBD_08_HU2_Shape.zip

In [5]:
# Call the decorator function using the @
@cached(
        # The func_key here will be wbd_08
        # this is for region 8 of the watershed boundary data
        'wbd_08')
# Create a new funtion, 3 parameters
def read_wbd_file(
    # Name of file to be downloaded and processed
    wbd_filename, 
    # Level of the data
    huc_level, 
    # Can further refine the cache_key
    cache_key):
    # Download and unzip
    # Define URL
    wbd_url = (
        "https://prd-tnm.s3.amazonaws.com"
        "/StagedProducts/Hydrography/WBD/HU2/Shape/"
        # Insert the name of the specific file wanted using f-string
        f"{wbd_filename}.zip")
    
    # Define, create nested path and dir for watershed boundaries
    # Path for watershed boundaries
    watershed_bd_dir = os.path.join(landcover_cluster_data_dir, 'watershed_bd')
    # Create directory for this path
    os.makedirs(watershed_bd_dir, 
            # If directory already exists no error is raised
            exist_ok=True)
    # Download and unzip the data into the above dir
    watershed_bd_dir = et.data.get_data(url=wbd_url)
                  
    # Read desired data
    # Path to shapefile in dir, subfolder, shapefile
    watershed_bd_path = os.path.join(watershed_bd_dir, 'Shape', f'WBDHU{huc_level}.shp')
    # Read shapefile as a gdf
    watershed_bd_gdf = gpd.read_file(watershed_bd_path, 
        # Use pyogrio library to read shapfile
        engine='pyogrio')
    
    # Return the gdf of the watershed boundaries
    return watershed_bd_gdf

In [6]:
# Apply the read_wbd_file function

# Define the huc_level - 12
huc_level = 12
# Download and open the shapefile
watershed_bd_gdf = read_wbd_file(
    "WBD_08_HU2_Shape", huc_level, cache_key=f'hu{huc_level}')

In [7]:
# Call gdf to see it
watershed_bd_gdf.head()

,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,states,...,name,hutype,humod,tohuc,noncontrib,noncontr_1,shape_Leng,shape_Area,ObjectID,geometry
0,{8AFB1AF9-7296-4303-89DE-14CD073B859A},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"535297,540579",29441.81,119.15,LA,...,Gourd Bayou-Youngs Bayou,S,"LE,ID,DD",080500011308,0.0,0.0,NaN,NaN,1,"POLYGON ((-92.00021 32.53586, -91.99994 32.535..."
1,{916A17A6-B4A0-4FD7-9BB8-FFD1936B15B2},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,535512,11406.67,46.16,LA,...,Hams Creek,S,ID,080802050104,0.0,0.0,NaN,NaN,2,"POLYGON ((-93.37574 30.58982, -93.3747 30.5891..."
2,{493C7EC1-2F1C-4B84-AFFB-6F6868A9868E},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"547190,559640",29138.21,117.92,LA,...,Caney Creek-Bayou D'Arbonne,S,NM,080402060503,0.0,0.0,NaN,NaN,3,"POLYGON ((-93.07761 32.88752, -93.07784 32.887..."
3,{49A3C087-B460-4F97-9D99-78CBB675248B},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"77417,78285",17759.39,71.87,AR,...,L'Aigle Creek-Saline River,S,NM,080402020206,0.0,0.0,NaN,NaN,4,"POLYGON ((-92.08947 33.29383, -92.0897 33.2938..."
4,{0FB41498-11EA-4AB1-AF05-E2A8E5E2E274},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,1628466,98564.62,398.88,LA,...,West Cote Blanche Bay,W,NM,080801030800,0.0,0.0,NaN,NaN,5,"POLYGON ((-91.62408 29.73947, -91.62195 29.737..."


In [8]:
# Define new variable for the plot
ms_delta_gdf = (
    # Filter the to the rows of watershed wanted
    watershed_bd_gdf[watershed_bd_gdf[f'huc{huc_level}']
    # Select watershed 
    .isin(['080902030506'])]
    # Merge geometries of all rows matching this watershed
    .dissolve()
)

# Call the gdf to see it 
ms_delta_gdf.head()

,geometry,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,...,huc12,name,hutype,humod,tohuc,noncontrib,noncontr_1,shape_Leng,shape_Area,ObjectID
0,"POLYGON ((-89.97047 29.74687, -89.96593 29.750...",{E942B72E-599E-48F5-908A-EA5265701C14},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"536881,539539",37355.86,151.17,...,080902030506,Manuel Canal-Spanish Lake,D,GC,080902030508,0.0,0.0,NaN,NaN,2560


In [9]:
# Plot site map of the watershed
ms_delta_site_map = (
(
    # Project to Mercator 
    ms_delta_gdf.to_crs(ccrs.Mercator())
    .hvplot(
        # Set geometry as slightly transparent and white
        alpha=.3, fill_color='white', 
        # Add ESRI tile background
        tiles='EsriImagery', 
        # Set basemap/background to Mercator to match
        crs=ccrs.Mercator(),
        # Set title
        title='Mississippi River Delta Watershed - Site Map',
        )
    .opts(
        # Size of plot
        width=600, height=300)
))

# Save the plot as html to be able to display online
hv.save(ms_delta_site_map, 'chi_chloropleth_asthma_by_tract.html')  

# Display the plot
ms_delta_site_map

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

# Site Description - Mississippi River Delta Watershed site Map
The Mississippi River Delta is located within the Lower Mississippi Valley (U.S.)
and has had much prior research done on land cover classes and change 
in land cover classes showing a wide range of types (water, developed, 
mechanically disturbed, mining, barren, forest, grass/shurb, agriculture, wetland) 
due to its location on land and near bodies of water (Karstensen and Sayler, 1-7). 
Historically it had deciduous forests, but after mass tree clearing through the 
1970's the land use converted to mostly agricultural and aquacultural use 
(Karstensen and Sayler, 1). Thus, these lands are tied to the economy providing 
numerous jobs; these lands are also home to a diversity of habitats for wildlife 
including endangered and threatened species whom are threatened by mass wetland 
loss and climate change (National Wildlife Federation). The watershed drains water 
from a large portion of the contiguous U.S. and eventually into the Gulf of 
Mexico (National Park Service, 2024). This is an important natural process that 
is being disrupted by humans resulting in the the river being disconnected from the 
floodplain (Morris, 2017). There are currently many ongoing efforts to 
restore the delta for pepople, wildlife, and protect from natural disaters (RESTORE 
The Mississippi River Delta). The work being done here is really inspiring and 
important - if you are curious to learn more I reccomend looking at the 
[Restore the Mississippi River Delta website](https://mississippiriverdelta.org) . 
The National Park Service also has a great 
[multifaceted report](https://www.nps.gov/locations/lowermsdeltaregion/the-natural-environment-the-delta-and-its-resources.htm) 
that goes over the history, geology, land uses, endangered and threatened species, 
national wildlife refuges, floods and extraordinary natural events, and challenges 
for the future - this would be a great source for further research and understanding 
the Mississippi River Delta. 

The site map above, if you zoom out, you get a better context of truely how 
close this is to the gulf. The latitude ranges less than 1 degree and 
the longitude ranges also around 1 degree. Visually this plot one can 
also see there seems to be elevation changes, a road or dam possibly to 
the left of the outlined area, and different types of land cover. For 
such a contained area it is visually diverse in land cover. It will be 
interesting to use satellite imagery to do analysis here because visually 
it is already diverse and this will only be added to with further analysis. 

## Citations 

* Karstensen, K.A., and Sayler, K.L., 2009, 
Land-cover change in the Lower Mississippi Valley, 1973–2000:
U.S. Geological Survey Open-File Report 2009–1280, 12 p.
https://pubs.usgs.gov/of/2009/1280/pdf/of2009-1280.pdf

* National Wildlife Federation. 'Mississippi River Delta'. 
https://www.nwf.org/Educational-Resources/Wildlife-Guide/Wild-Places/Mississippi-River-Delta

* National Park Service. 'Mississippi River Facts'. 
Updated December 31, 2024.
https://www.nps.gov/miss/riverfacts.htm#:~:text=Some%20like%20to%20measure%20the,(2.7%20million%20square%20miles).

* Morris, Christopher. Environmental Science, 'Environmental History of the 
Mississippi River and Delta'. September 26, 2017.
https://oxfordre.com/environmentalscience/display/10.1093/acrefore/9780199389414.001.0001/acrefore-9780199389414-e-114?p=emailAke4d9oJyr02w&d=/10.1093/acrefore/9780199389414.001.0001/acrefore-9780199389414-e-114#:~:text=The%20Mississippi%20River%20delta%20begins,approach%20to%20Mississippi%20River%20management.

* RESTORE The Mississippi River Delta. 'America Needs the Delta'.
https://mississippiriverdelta.org


* National Park Service -  Lower Mississippi Delta Region. 
'The Natural Environment: The Delta and Its Resources'. 
Updated November 16, 2017.
https://www.nps.gov/locations/lowermsdeltaregion/the-natural-environment-the-delta-and-its-resources.htm


* Watershed Boundary Dataset About- https://www.usgs.gov/national-hydrography/access-national-hydrography-products

## STEP 3: MULTISPECTRAL DATA

### Search for data

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Log in to the <code>earthaccess</code> service using your Earthdata
credentials:
<code>python      earthaccess.login(persist=True)</code></li>
<li>Modify the following sample code to search for granules of the
HLSL30 product overlapping the watershed boundary from May to October
2023 (there should be 76 granules):
<code>python      results = earthaccess.search_data(          short_name="...",          cloud_hosted=True,          bounding_box=tuple(gdf.total_bounds),          temporal=("...", "..."),      )</code></li>
</ol></div></div>

In [11]:
# Log in to earthaccess
earthaccess.login(
    # Prompt to input login credentials
    strategy = "interactive", 
    # Remember credentials
    persist = True)

# Search for HLS tiles
hls_results = earthaccess.search_data(
    # Filter search for data only related to HLS30
    # HLS30 is Harmonized Landsat Sentinal data 30m resolution
    short_name = "HLSL30",
    # Only return cloud hosted data
    cloud_hosted = True,
    # Set spatial bounds
    # Convert bounding box into tuple format for the search
    bounding_box = tuple(
        # Give min and max geogrpahic bounds of the gdf
        ms_delta_gdf.total_bounds),
    # Set temporal bounds
    temporal=("2024-06", "2024-08")
)

# Call the variable to see if that worked
hls_results

[Collection: {'EntryTitle': 'HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -89.79864173, 'Latitude': 29.70347853}, {'Longitude': -89.76643746, 'Latitude': 30.69278312}, {'Longitude': -90.91181412, 'Latitude': 30.71627038}, {'Longitude': -90.93262544, 'Latitude': 29.72659663}, {'Longitude': -89.79864173, 'Latitude': 29.70347853}]}}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-06-07T16:31:11.509Z', 'EndingDateTime': '2024-06-07T16:31:11.509Z'}}
 Size(MB): 169.50417041778564
 Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B10.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.SAA.tif', 'https://data.l

### Compile information about each granule

I recommend building a GeoDataFrame, as this will allow you to plot the
granules you are downloading and make sure they line up with your
shapefile. You could also use a DataFrame, dictionary, or a custom
object to store this information.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>For each search result:
<ol type="1">
<li>Get the following information (HINT: look at the [‘umm’] values for
each search result):
<ul>
<li>granule id (UR)</li>
<li>datetime</li>
<li>geometry (HINT: check out the shapely.geometry.Polygon class to
convert points to a Polygon)</li>
</ul></li>
<li>Open the granule files. I recomment opening one granule at a time,
e.g. with (<code>earthaccess.open([result]</code>).</li>
<li>For each file (band), get the following information:
<ul>
<li>file handler returned from <code>earthaccess.open()</code></li>
<li>tile id</li>
<li>band number</li>
</ul></li>
</ol></li>
<li>Compile all the information you collected into a GeoDataFrame</li>
</ol></div></div>

In [7]:
#
def get_earthaccess_links(results):
    #
    url_re = re.compile(
        # Create a regular expression
        r'\.(?P<tile_id>\w+)\.\d+T\d+\.v\d\.\d\.(?P<band>[A-Za-z0-9]+)\.tif')

    # Loop through each granule
    # Create empty list
    link_rows = []
    #
    url_dfs = []
    # Start for loop
    for granule in tqdm(results):
        # Get granule information
        info_dict = granule['umm']
        granule_id = info_dict['GranuleUR']
        datetime = pd.to_datetime(
            info_dict
            ['TemporalExtent']['RangeDateTime']['BeginningDateTime'])
        points = (
            info_dict
            ['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['GPolygons'][0]
            ['Boundary']['Points'])
        geometry = Polygon(
            [(point['Longitude'], point['Latitude']) for point in points])
        
        # Get URL - 
        files = earthaccess.open([granule])

        # Build metadata DataFrame
        for file in files:
            # Apply regular expression
            match = url_re.search(
                # to the file.full_name
                file.full_name)
            # Conditional statement
            # If match is found this code block will execute
            if match is not None:
                #
                link_rows.append(
                    #
                    gpd.GeoDataFrame(
                        # Create a dictionary with keys as columns
                        dict(
                            # Column names:
                            # Extract datetime from file name
                            datetime=[datetime],
                            # Extract tile id from file name
                            tile_id=[match.group('tile_id')],
                            # Extract band from file name
                            band=[match.group('band')],
                            # 
                            url=[file],
                            # Extract geomentry from the file
                            geometry=[geometry]
                        ),
                        # Specify CRS for gdf based on lat lon
                        crs="EPSG:4326"
                    )
                )

    # Concatenate metadata DataFrame
    #
    file_df = pd.concat(
        # Reset index to a default integer index
        # This is because each gdf might have its own index which 
        # would make it hard to continue doing analysis on the data
        link_rows).reset_index(
            # Drop the old index column
            # This keeps the old index column from being
            # added as an additional column 
            drop=True)
    #
    return file_df

### Open, crop, and mask data

This will be the most resource-intensive step. I recommend caching your
results using the `cached` decorator or by writing your own caching
code. I also recommend testing this step with one or two dates before
running the full computation.

This code should include at least one **function** including a
numpy-style docstring. A good place to start would be a function for
opening a single masked raster, applying the appropriate scale
parameter, and cropping.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>For each granule:
<ol type="1">
<li><p>Open the Fmask band, crop, and compute a quality mask for the
granule. You can use the following code as a starting point, making sure
that <code>mask_bits</code> contains the quality bits you want to
consider: ```python # Expand into a new dimension of binary bits bits =
( np.unpackbits(da.astype(np.uint8), bitorder=‘little’)
.reshape(da.shape + (-1,)) )</p>
<p># Select the required bits and check if any are flagged mask =
np.prod(bits[…, mask_bits]==0, axis=-1) ```</p></li>
<li><p>For each band that starts with ‘B’:</p>
<ol type="1">
<li>Open the band, crop, and apply the scale factor</li>
<li>Name the DataArray after the band using the <code>.name</code>
attribute</li>
<li>Apply the cloud mask using the <code>.where()</code> method</li>
<li>Store the DataArray in your data structure (e.g. adding a
GeoDataFrame column with the DataArray in it. Note that you will need to
remove the rows for unused bands)</li>
</ol></li>
</ol></li>
</ol></div></div>

In [9]:
# Loop through each image

    # Open granule cloud cover

    # Compute cloud mask

    # Loop through each spectral band

        # Open, crop, and mask the band

        # Add the DataArray to the metadata DataFrame row

    # Reassemble the metadata DataFrame

### Merge and Composite Data

You will notice for this watershed that: 1. The raster data for each
date are spread across 4 granules 2. Any given image is incomplete
because of clouds

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li><p>For each band:</p>
<ol type="1">
<li><p>For each date:</p>
<ol type="1">
<li>Merge all 4 granules</li>
<li>Mask any negative values created by interpolating from the nodata
value of -9999 (<code>rioxarray</code> should account for this, but
doesn’t appear to when merging. If you leave these values in they will
create problems down the line)</li>
</ol></li>
<li><p>Concatenate the merged DataArrays along a new date
dimension</p></li>
<li><p>Take the mean in the date dimension to create a composite image
that fills cloud gaps</p></li>
<li><p>Add the band as a dimension, and give the DataArray a
name</p></li>
</ol></li>
<li><p>Concatenate along the band dimension</p></li>
</ol></div></div>

In [11]:
# Merge and composite and image for each band

        # Merge granules for each date

        # Mask negative values

    # Composite images across dates

## STEP 4: K-MEANS

Cluster your data by spectral signature using the k-means algorithm.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Convert your DataArray into a <strong>tidy</strong> DataFrame of
reflectance values (hint: check out the <code>.to_dataframe()</code> and
<code>.unstack()</code> methods)</li>
<li>Filter out all rows with no data (all 0s or any N/A values)</li>
<li>Fit a k-means model. You can experiment with the number of groups to
find what works best.</li>
</ol></div></div>

In [13]:
# Convert spectral DataArray to a tidy DataFrame

# Running the fit and predict functions at the same time.
# We can do this since we don't have target data.

# Add the predicted values back to the model DataFrame

## STEP 5: PLOT

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Create a plot that shows the k-means clusters next to an RGB image of
the area. You may need to brighten your RGB image by multiplying it by
10. The code for reshaping and plotting the clusters is provided for you
below, but you will have to create the RGB plot yourself!</p>
<p>So, what is <code>.sortby(['x', 'y'])</code> doing for us? Try the
code without it and find out.</p></div></div>

In [15]:
# Plot the k-means clusters
(
    rgb_plot
    + 
    model_df.clusters.to_xarray().sortby(['x', 'y']).hvplot(
        cmap="Colorblind", aspect='equal') 
)

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Don’t forget to interpret your plot!</p></div></div>

**YOUR PLOT HEADLINE AND DESCRIPTION HERE**